# 3-1. Premade Estimators

본 실습에서는 Estimators를 사용하여 TensorFlow의 Iris classification 문제를 해결하는 방법을 설명합니다.

Estimator는 TensorFlow의 완전한 Model의 높은 수준의 표현으로, 간편한 확장 및 비동기식 교육을 위해 설계되었습니다.

TensorFlow 2.0에서는 Keras API가 이와 같은 많은 작업을 수행할 수 있으며, 보다 쉽게 배울 수 있는 API로 여겨집니다.

새로 시작하는 경우 Keras부터 시작하는 것이 좋습니다.

## 1. First things first

시작하기 위해 먼저 TensorFlow와 필요한 여러 라이브러리를 가져옵니다.

In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

import pprint

***

## 2. The data set

이 문서의 샘플 프로그램은 Iris 꽃들을 그들의 꽃받침과 꽃잎의 크기에 따라 3개의 다른 종으로 분류하는 모델을 만들고 테스트합니다.

Iris 데이터 세트를 사용하여 모델을 train 합니다. 

***

Iris 데이터 세트에는 4가지 feature와 1개의 label이 있습니다. 

4가지 feature는 개별 Iris 꽃의 다음과 같은 식물학적 특성을 식별합니다.

   * sepal length (꽃받침 길이)
   * sepal width (꽃받침 너비)
   * petal length (꽃잎 길이)
   * petal width (꽃잎 너비)
   
이 정보를 기반으로 데이터를 구문 분석하는 데 유용한 상수를 몇 개 정의할 수 있습니다.

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

다음으로, Keras와 Pandas를 사용하여 Iris 데이터 세트를 다운로드하고 구문 분석합니다. 

training 및 testing을 위해 별도의 데이터 세트를 보관합니다.

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


데이터를 검사하여 4개의 float feature columns과 1개의 int32 label이 있는지 확인할 수 있습니다.

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


각 데이터 세트에 대해 labels를 분할하여 모델이 predict하도록 train합니다.

In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


***

## 3. Overview of programming with Estimators

이제 데이터가 설정되었으므로 TensorFlow Estimator를 사용하여 모델을 정의할 수 있습니다.

Estimator는 ```tf.estimator.Estimator```에서 파생된 클래스입니다.

TensorFlow는 일반적인 ML 알고리즘을 구현하기 위한 ```tf.estimator```(예: ```LinearRegressor```) collection을 제공합니다.

그 외에도 own custom Estimators를 작성할 수 있습니다.

***

처음 시작할 때는 pre-made Estimators를 사용하는 것이 좋습니다.

pre-made Estimators를 기반으로 TensorFlow 프로그램을 작성하려면 다음 작업을 수행해야 합니다.

   * 하나 이상의 input function을 생성합니다.
   
   * model's feature columns를 정의합니다.
   
   * feature columns와 다양한 hyperparameters를 지정하여 Estimator를 인스턴스화합니다.
   
   * 데이터 소스로 적절한 input function을 전달하여 Estimator object에서 하나 이상의 method를 호출합니다.
   
Iris classification을 위해 이러한 작업이 어떻게 구현되는지 살펴보겠습니다.

***

## 4. Create input functions

training, evaluating 및 prediction을 위한 데이터를 제공하기 위한 input functions를 생성해야 합니다.

input function는 ```tf.data.Dataset``` 객체를 반환하여 다음의 two-element tuple을 출력하는 함수입니다.

   * features - Python dictionary 이며, 다음과 같습니다.
      * 각 key는 feature의 이름입니다.
      * 각 value는 해당 feature의 values를 모두 포함하는 array입니다.
      
   * label - 모든 example에 대한 values of the label을 포함하는 array입니다.
   
다음은 input function의 format을 보여 주는 간단한 구현 방법입니다.

In [7]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

input function은 원하는 대로 features dictionary 및 label list를 생성할 수 있습니다. 

그러나 모든 종류의 데이터를 구문 분석할 수 있는 TensorFlow의 Dataset API를 사용하는 것이 좋습니다.

Dataset API는 많은 일반적인 사례를 처리할 수 있습니다. 

예를 들어, Dataset API를 사용하면 대용량 collection으로부터 records를 쉽게 읽고 동시에 이를 single stream에 결합할 수 있습니다.

***

이 예에서는 pandas를 사용하여 데이터를 로드하고 in-memory data로 input pipeline을 구축하려고 합니다.

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


***

## 5. Define the feature columns

feature column은 모델에서 features dictionary의 raw input data를 사용하는 방법을 설명하는 object입니다.

Estimator model을 build할 때, 모형에 사용하기를 원하는 각 features를 설명하는 list of feature columns을 이 모델에 전달합니다.

```tf.feature_column``` 모듈은 model에 데이터를 나타내는 여러 가지 옵션을 제공합니다.

***

Iris의 경우, 4개의 raw features는 numeric values이므로, 

Estimator mode에 4개의 features를 각각 32비트 부동 소수점(32-bit floating-point) 값으로 나타내도록 지시하는 list of feature columns을 build합니다.

따라서 feature column을 생성할 코드는 다음과 같습니다.

In [9]:
# Feature columns describe how to use the input.
print(train.keys())

my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
pprint.pprint(my_feature_columns)

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


feature columns는 여기서 보여드리는 것보다 훨씬 더 정교할 수 있습니다.

이제 model이 raw features를 나타내는 방법에 대한 설명을 얻었으므로, estimator를 작성할 수 있습니다.

***

## 6. Instantiate an estimator

Iris 문제는 전형적인 classification 문제입니다.

다행히도 TensorFlow는 다음과 같은 몇 가지 pre-made classifier Estimators를 제공합니다.

   * ```tf.estimator.DNNClassifier``` - multi-class classification를 수행하는 deep models을 위한 것입니다.

   * ```tf.estimator.DNNLinearCombinedClassifier``` - wide & deep models을 위한 것입니다.

   * ```tf.estimator.LinearClassifier``` - linear(선형) models을 기반으로 하는 classifiers를 위한 것입니다.


Iris 문제의 경우 ```tf.estimator.DNNClassifier```가 최선의 선택인 것 같습니다.

다음은 이 Estimator를 인스턴스화하는 방법입니다.

In [16]:
print('my_feature_columns : \n')
pprint.pprint(my_feature_columns)
print()

# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

print()
print(classifier)

my_feature_columns : 

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8d/j2dvm8_n3wv_z7r6r99z3_kw0000gn/T/tmpjc_9xsji', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eva

***

## 7. Train, Evaluate, and Predict


이제 Estimator object가 있으므로 methods를 호출하여 다음 작업을 수행할 수 있습니다.

   * Train the model.

   * Evaluate the trained model.

   * Use the trained model to make predictions.

***

### Train the model


다음과 같이 Estimator's train method를 호출하여 모델을 train합니다.

In [20]:
# Train the Model.

'''
def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)
'''

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8d/j2dvm8_n3wv_z7r6r99z3_kw0000gn/T/tmpjc_9xsji/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/8d/j2dvm8_n3wv_z7r6r

lambda로 ```input_fn```을 호출하여 인자를 캡처하는 동시에 

Estimator에서 예상한 대로 인자를 사용하지 않는 input function을 제공합니다.

```steps``` 인자는 여러 training steps가 끝난 후 training을 중지하는 방법을 설명합니다.

***

### Evaluate the trained model

이제 모델이 train 했으므로 성능에 대한 몇 가지 통계를 얻을 수 있습니다. 

다음 코드 블록은 test data에 대해 학습된 모델의 accuracy(정확도)를 평가합니다.

In [21]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-05T11:14:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8d/j2dvm8_n3wv_z7r6r99z3_kw0000gn/T/tmpjc_9xsji/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-05-11:14:29
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.93333334, av

train method에 대한 호출과 달리, evaluate에 대한 steps argument는 통과하지 않았습니다.

evaluate에 대한 ```input_fn```은 단 하나의 데이터만 생성합니다.

```eval_result``` dictionary에는 

   * average_loss(mean loss per sample)
   * the loss (mean loss per mini-batch)
   * estimator's global_step 값(training 반복 횟수)
   
등이 포함되어 있습니다.

***

### Making predictions (inferring) from the trained model

이제 좋은 평가 결과를 만들어내는 훈련된 모델을 갖게 되었습니다.

이제 몇몇 라벨이 붙어 있지 않은 측정(unlabeled measurements)을 바탕으로 Iris 꽃의 종을 예측하기 위해 훈련된 모델을 사용할 수 있습니다.

training 및 evaluation과 마찬가지로, 단일 함수 호출(single function call)을 사용하여 prediction해야 합니다.

In [22]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

The ```predict``` method는 Python iterable을 반환하여, 각 example에 대한 예측 결과 dictionary를 생성합니다. 

다음 코드는 몇 가지 predictions와 그 확률을 출력합니다.

In [23]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8d/j2dvm8_n3wv_z7r6r99z3_kw0000gn/T/tmpjc_9xsji/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (77.7%), expected "Setosa"
Prediction is "Versicolor" (51.4%), expected "Versicolor"
Prediction is "Virginica" (63.8%), expected "Virginica"
